Function to rename all files in folder/subfolder, replacing old_string with new_string

In [17]:
import os
import re
import sys

def get_file_size(file_path):
    return os.path.getsize(file_path)

def get_next_available_filename(file_path):
    base, ext = os.path.splitext(file_path)
    counter = 1
    while os.path.exists(file_path):
        file_path = f"{base}({counter}){ext}"
        counter += 1
    return file_path

def prompt_for_action(old_file, new_file):
    old_size = get_file_size(old_file)
    new_size = get_file_size(new_file)
    
    print(f"\nConflict: '{new_file}' already exists.")
    print(f"Old file size: {old_size} bytes")
    print(f"Existing file size: {new_size} bytes")
    
    while True:
        choice = input("Choose action: [o]verwrite, [k]eep both, [s]kip: ").lower()
        if choice in ['o', 'k', 's']:
            return choice
        print("Invalid choice. Please try again.")

def normalize_path(path):
    # Convert Windows backslashes to forward slashes
    path = path.replace('\\', '/')
    # Normalize the path
    return os.path.normpath(path)

def rename_fn(folder_path, old_string, new_string):
    print(f"Starting rename operation in folder: {folder_path}")
    print(f"Replacing '{old_string}' with '{new_string}'")

    # Normalize the path for cross-platform compatibility
    folder_path = normalize_path(folder_path)
    print(f"Normalized folder path: {folder_path}")

    old_string_escaped = re.escape(old_string)
    print(f"Escaped search string: {old_string_escaped}")

    file_count = 0
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if re.search(old_string_escaped, filename):
                old_file_path = os.path.join(root, filename)
                new_filename = re.sub(old_string_escaped, new_string, filename)
                new_file_path = os.path.join(root, new_filename)
                
                try:
                    if os.path.exists(new_file_path):
                        action = prompt_for_action(old_file_path, new_file_path)
                        if action == 'o':
                            os.remove(new_file_path)
                        elif action == 'k':
                            new_file_path = get_next_available_filename(new_file_path)
                        else:  # skip
                            print(f"Skipped: {old_file_path}")
                            continue
                    
                    print(f"Renaming: {old_file_path} -> {new_file_path}")
                    os.rename(old_file_path, new_file_path)
                    print(f"Successfully renamed: {old_file_path} -> {new_file_path}")
                    file_count += 1
                except Exception as e:
                    print(f"Error renaming {old_file_path}: {str(e)}")

    print(f"Rename operation complete. {file_count} files renamed.")

# Example usage:
# For Mac/Linux:
# rename_fn('/Volumes/', 'video - ', '')
# For Windows:
# rename_fn(r'C:\Program Files\My Folder', 'old-', 'new-')